# 1. Web scraping

In [2]:
import requests
import bs4
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models import Doc2Vec
from collections import namedtuple
import gensim.utils
from langdetect import detect
import re
import string

In [3]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

In [2]:
url='https://www.goodreads.com/list/show/1043.Books_That_Should_Be_Made_Into_Movies?page='

In [3]:
titles=[]
authors=[]
#ratings=[]
for page in range(1,101):

    # Concatenate to get new page URL
    page_url = url+str(page)
    # Obtain Request
    res = requests.get(page_url)
    # Turn into Soup
    soup = bs4.BeautifulSoup(res.text,'lxml')
    # Add Authors to our set
    for name in soup.select(".bookTitle"):
        titles.append(name.text)
    for name_au in soup.select(".authorName"):
        authors.append(name_au.text)
   #for rt in soup.select(".minirating"):
  #      ratings.append(rt.text)

In [109]:
book=pd.DataFrame(data=(titles,authors)).transpose()
book1=book.iloc[:10000]
book1.columns=['title','author']
book1['title']=book1['title'].str.replace('\n','')
book1.to_csv('book1.csv')

C:\Users\vutha\AppData\Local\Temp\ipykernel_3412\733938569.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book1['title']=book1['title'].str.replace('\n','')


# 2. TFIDF

In [4]:
df=pd.read_csv('book1.csv',index_col=0)

In [5]:
df_news=df
df_news['title']=[entry.lower() for entry in df_news['title']]
df_news['author']=[entry.lower() for entry in df_news['author']]

In [9]:
df_news['title']=df_news.title.str.replace("(","")
df_news['title']=df_news.title.str.replace(")","")
df_news['title']=df_news.title.str.replace("#","")
df_news['title']=df_news.title.str.replace(",","")
for i in range(len(df_news['title'])):
    for ele in df_news['title'][i]:
        if ele.isdigit():
            df_news['title'][i]=df_news['title'][i].replace(ele,"")

C:\Users\vutha\AppData\Local\Temp\ipykernel_19156\2960310958.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_news['title']=df_news.title.str.replace("(","")
C:\Users\vutha\AppData\Local\Temp\ipykernel_19156\2960310958.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_news['title']=df_news.title.str.replace(")","")


In [10]:
df_news

,title,author
0,city of ashes the mortal instruments,cassandra clare
1,hush hush hush hush,becca fitzpatrick
2,city of glass the mortal instruments,cassandra clare
3,clockwork angel the infernal devices,cassandra clare
4,marked house of night,p.c. cast
...,...,...
9995,the fallen boys,michael english
9996,the noah narratives book i daughter of the cov...,elaine d walsh
9997,the ripple in space-time,sara webb quest (s.w. quest)
9998,the devil's grin,mackenzie knight


In [61]:
stop_words = set(stopwords.words('english'))
for i, entry in enumerate(df['title']):
    word_tokens = word_tokenize(entry)
  
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    
    df_news['clean_word'][i]=" ".join(filtered_sentence)

In [62]:
df_news

,title,author,clean_word
0,city of ashes the mortal instruments,cassandra clare,city ashes mortal instruments
1,hush hush hush hush,becca fitzpatrick,hush hush hush hush
2,city of glass the mortal instruments,cassandra clare,city glass mortal instruments
3,clockwork angel the infernal devices,cassandra clare,clockwork angel infernal devices
4,marked house of night,p.c. cast,marked house night
...,...,...,...
9995,the fallen boys,michael english,fallen boys
9996,the noah narratives book i daughter of the cov...,elaine d walsh,noah narratives book daughter covenant noah na...
9997,the ripple in space-time,sara webb quest (s.w. quest),ripple space-time
9998,the devil's grin,mackenzie knight,devil 's grin


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
## Create Vocabulary
vocabulary = set()
for doc in df_news.clean_word:
    vocabulary.update(doc.split())
vocabulary = list(vocabulary)
# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary)

# Fit the TfIdf model
tfidf.fit(df_news.clean_word)
# Transform the TfIdf model
tfidf_tran=tfidf.transform(df_news.clean_word)

In [67]:
import numpy as np
from numpy.linalg import norm
 
def cosine_sim(a, b):
    cosine = np.dot(a,b)/(norm(a)*norm(b))
    return cosine


In [129]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine_similarity_T(k, query):
    new_query=query.lower()
    new_query=[w for w in query.split() if not w.lower() in stop_words]
    
    query_vector = tfidf.transform(new_query)
    
    cosine_distance = cosine_similarity(query_vector, tfidf_tran)
    
    similarity_list = cosine_distance[0]
                    
    out = np.array(similarity_list).argsort()[-k:][::-1]
    #print("")
    similarity_list.sort()
    result = pd.DataFrame()
    for i,index in enumerate(out):
        result.loc[i,'index'] = str(index)
        result.loc[i,'title'] = df_news['title'][index]
    for j,simScore in enumerate(similarity_list[-k:][::-1]):
        result.loc[j,'Score'] = simScore
    return result


In [133]:
cosine_similarity_T(10,'city')

,index,title,Score
0,3450,city,1.000000
1,1364,the city & the city,1.000000
2,7538,black city black city,0.744756
3,8049,summer in the city,0.696806
4,4139,the city and the stars,0.685815
5,8238,immortal city immortal city,0.680224
6,9556,the city not long after,0.652969
7,492,city of thieves,0.627354
8,6315,city of dark magic city of dark magic,0.621866
9,2362,city of skies,0.615394
